<a href="https://colab.research.google.com/github/pandearthr/learn-pandas/blob/main/uts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
df['lang'].value_counts()


,count
lang,
in,791


In [34]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.0 MB/s eta 0:00:00


In [35]:
from google.colab import files
uploaded = files.upload()


Saving baju_thriftingg.csv to baju_thriftingg (1).csv


In [36]:
df = pd.read_csv(list(uploaded.keys())[0], encoding='latin-1')
df.head()


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username;;;;;;;
0,1986646440144478253,Fri Nov 07 04:06:36 +0000 2025,0,Menteri Perdagangan (Mendag) Budi Santoso buka...,1986646440144478253,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/198664644014447...,4755284968,;;;;;;;
1,1986640662880788584,Fri Nov 07 03:45:45 +0000 2025,0,@sjwsantun ayo kita jadwalkan thrifting baju k...,1986641193246343641,NaN,sjwsantun,in,NaN,0,1,0,https://x.com/undefined/status/198664119324634...,941243650054406144,;;;;;;;
2,1986498987701248404,Thu Nov 06 18:20:40 +0000 2025,0,Era thriting di market place Indonesia berakhi...,1986498987701248404,NaN,NaN,in,NaN,0,1,0,https://x.com/undefined/status/198649898770124...,537442450,;;;;;;;
3,1986479639930544149,Thu Nov 06 17:03:47 +0000 2025,0,dapet baju bagus di pasar loak tuh sesenang it...,1986479639930544149,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/198647963993054...,1289197162790711297,;;;;;;;
4,1986453130041864479,Thu Nov 06 15:18:27 +0000 2025,0,Lagi jual thrifting kyk jual baju distro modal...,1986453130041864479,NaN,NaN,in,NaN,0,0,0,https://x.com/undefined/status/198645313004186...,256008602,;;;;;;;


In [37]:
print("Jumlah Data:", len(df))
print("\nNama Kolom:\n", df.columns)
print("\nCek Missing Value:\n", df.isnull().sum())
print("\nSample Tweet:\n", df['full_text'].head(10))
df['lang'].value_counts()


Jumlah Data: 791

Nama Kolom:
 Index(['conversation_id_str', 'created_at', 'favorite_count', 'full_text',
       'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location',
       'quote_count', 'reply_count', 'retweet_count', 'tweet_url',
       'user_id_str', 'username;;;;;;;'],
      dtype='object')

Cek Missing Value:
 conversation_id_str          0
created_at                   0
favorite_count               0
full_text                    0
id_str                       0
image_url                  671
in_reply_to_screen_name    468
lang                         0
location                   791
quote_count                  0
reply_count                  0
retweet_count                0
tweet_url                    0
user_id_str                  0
username;;;;;;;              1
dtype: int64

Sample Tweet:
 0    Menteri Perdagangan (Mendag) Budi Santoso buka...
1    @sjwsantun ayo kita jadwalkan thrifting baju k...
2    Era thriting di market place Indonesia berakhi...
3    d

,count
lang,
in,791


In [38]:
positif = ['bagus','murah','suka','puas','mantap','kualitas','nyaman',
           'recommended','rekomen','worth','senang','cakep','menyenangkan']

negatif = ['jelek','bau','sobek','kecewa','buruk','mahal','kapok','tidak sesuai',
           'parah','risih','tipu','nyesel','nipu']

def lexicon_label(text):
    text = str(text).lower()
    score = 0
    for w in positif:
        if w in text:
            score += 1
    for w in negatif:
        if w in text:
            score -= 1
    if score > 0:
        return 'Positif'
    elif score < 0:
        return 'Negatif'
    else:
        return 'Netral'

df['sentimen_lexicon'] = df['full_text'].apply(lexicon_label)
df[['full_text','sentimen_lexicon']].head(20)


,full_text,sentimen_lexicon
0,Menteri Perdagangan (Mendag) Budi Santoso buka...,Netral
1,@sjwsantun ayo kita jadwalkan thrifting baju k...,Netral
2,Era thriting di market place Indonesia berakhi...,Netral
3,dapet baju bagus di pasar loak tuh sesenang it...,Positif
4,Lagi jual thrifting kyk jual baju distro modal...,Netral
5,org2 bacot bgt masalah thrifting trs yg di poi...,Netral
6,@mistangelical Cayang terima kasih banyak! &lt...,Netral
7,@naxeon jadi mana baju hasil thrifting yg mami...,Netral
8,even liat mereka beli baju di store aja vibes ...,Netral
9,Purbaya Larang Impor Baju Bekas Ecommerce Take...,Netral


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('sentimen_lexicon').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('index', ascending=True)
for i, (series_name, series) in enumerate(df_sorted.groupby('sentimen_lexicon')):
  _plot_series(series, series_name, i)
  fig.legend(title='sentimen_lexicon', bbox_to_anchor=(1, 1), loc='upper left')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_3['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_4['sentimen_lexicon'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_4, x='index', y='sentimen_lexicon', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [45]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stopwords = set(open('/content/stopword.txt').read().split())

stopwords = set([
    'yang','dan','di','ke','dari','itu','ini','untuk','dengan','ada','karena',
    'pada','saat','jadi','sudah','atau','kamu','aku','saya'
])

In [46]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)   # remove URL
    text = re.sub(r'@\w+', '', text)                    # remove mentions
    text = re.sub(r'[^a-zA-Z ]', ' ', text)             # remove symbols
    text = ' '.join([w for w in text.split() if w not in stopwords])  # stopwords
    text = stemmer.stem(text)                           # stemming
    return text


In [56]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

cache = {}

def stem_word_cached(word):
    if word not in cache:
        cache[word] = stemmer.stem(word)
    return cache[word]


In [57]:
df['clean_text'] = df['full_text'].apply(preprocess)
df[['full_text','clean_text']].head()


,full_text,clean_text
0,Menteri Perdagangan (Mendag) Budi Santoso buka...,menteri dagang mendag budi santoso buka suara ...
1,@sjwsantun ayo kita jadwalkan thrifting baju k...,ayo kita jadwal thrifting baju kerja itu
2,Era thriting di market place Indonesia berakhi...,era thriting di market place indonesia akhir t...
3,dapet baju bagus di pasar loak tuh sesenang it...,dapet baju bagus di pasar loak tuh senang ituu...
4,Lagi jual thrifting kyk jual baju distro modal...,lagi jual thrifting kyk jual baju distro modal...


from matplotlib import pyplot as plt
import seaborn as sns
_df_5.groupby('full_text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_6.groupby('clean_text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['clean_text'].value_counts()
    for x_label, grp in _df_7.groupby('full_text')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('full_text')
_ = plt.ylabel('clean_text')

In [60]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# --- Lexicon-based Sentiment Analysis setup (from previous cells) ---
positif = ['bagus','murah','suka','puas','mantap','kualitas','nyaman',
           'recommended','rekomen','worth','senang','cakep','menyenangkan']

negatif = ['jelek','bau','sobek','kecewa','buruk','mahal','kapok','tidak sesuai',
           'parah','risih','tipu','nyesel','nipu']

def lexicon_label(text):
    text = str(text).lower()
    score = 0
    for w in positif:
        if w in text:
            score += 1
    for w in negatif:
        if w in text:
            score -= 1
    if score > 0:
        return 'Positif'
    elif score < 0:
        return 'Negatif'
    else:
        return 'Netral'

# --- Text Preprocessing setup (from previous cells) ---
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stopwords definition from previous cell
stopwords = set([
    'yang','dan','di','ke','dari','itu','ini','untuk','dengan','ada','karena',
    'pada','saat','jadi','sudah','atau','kamu','aku','saya'
])

cache = {}

def stem_word_cached(word):
    if word not in cache:
        cache[word] = stemmer.stem(word)
    return cache[word]

def preprocess(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)   # remove URL
    text = re.sub(r'@\w+', '', text)                    # remove mentions
    text = re.sub(r'[^a-zA-Z ]', ' ', text)             # remove symbols
    text = ' '.join([w for w in text.split() if w not in stopwords])  # stopwords
    text = stemmer.stem(text)                           # stemming
    return text

# Create the 'sentimen_lexicon' and 'clean_text' columns
df['sentimen_lexicon'] = df['full_text'].apply(lexicon_label)
df['clean_text'] = df['full_text'].apply(preprocess)

# Original content of the cell
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['clean_text'])

y = df['sentimen_lexicon']


In [61]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['clean_text'])

y = df['sentimen_lexicon']


In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [63]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
pred_nb = nb.predict(X_test)

print("=== NAIVE BAYES ===")
print("Akurasi:", accuracy_score(y_test, pred_nb))
print(classification_report(y_test, pred_nb))


=== NAIVE BAYES ===
Akurasi: 0.7610062893081762
              precision    recall  f1-score   support

     Negatif       0.00      0.00      0.00        12
      Netral       0.76      1.00      0.86       121
     Positif       0.00      0.00      0.00        26

    accuracy                           0.76       159
   macro avg       0.25      0.33      0.29       159
weighted avg       0.58      0.76      0.66       159



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [64]:
svm = LinearSVC()
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)

print("=== SVM ===")
print("Akurasi:", accuracy_score(y_test, pred_svm))
print(classification_report(y_test, pred_svm))


=== SVM ===
Akurasi: 0.8679245283018868
              precision    recall  f1-score   support

     Negatif       1.00      0.25      0.40        12
      Netral       0.86      0.99      0.92       121
     Positif       0.88      0.58      0.70        26

    accuracy                           0.87       159
   macro avg       0.92      0.61      0.67       159
weighted avg       0.88      0.87      0.85       159



In [66]:
df.to_csv("baju_thriftingg_labeled.csv", index=False)

from google.colab import files
files.download("baju_thriftingg_labeled.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>